In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/insurance/insurance.csv


In [2]:
df = pd.read_csv('/kaggle/input/insurance/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error

# Veriyi oku
df = pd.read_csv('/kaggle/input/insurance/insurance.csv')

# Eksik veri var mı?
print("Eksik Veriler:\n", df.isnull().sum())

Eksik Veriler:
 age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


In [8]:
# Kategorik ve sayısal sütunlar
cat_cols = df.select_dtypes(include='object').columns.tolist()
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Outlier Analizi (örnek olarak BMI değişkeni)
q1 = df["bmi"].quantile(0.25)
q3 = df["bmi"].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = df[(df["bmi"] < lower_bound) | (df["bmi"] > upper_bound)]
print(f"\nOutlier sayısı (bmi): {len(outliers)}")


Outlier sayısı (bmi): 9


In [11]:
# Feature-Target ayır
X = df.drop("charges", axis=1)
y = df["charges"]

# Veri bölme
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=42)

In [12]:
# Pipeline: kategorik değişkenler için encoding
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first"), cat_cols)
    ],
    remainder="passthrough"
)

# Model seti
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42)
}

results = []

# Model eğitimi ve değerlendirme
for name, model in models.items():
    pipeline = Pipeline([
        ("preprocess", preprocessor),
        ("model", model)
    ])
    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_val)

    rmse = np.sqrt(mean_squared_error(y_val, preds))
    try:
        rmsle = np.sqrt(mean_squared_log_error(y_val, np.maximum(preds, 0)))
    except:
        rmsle = None

    results.append((name, rmse, rmsle))

# Sonuçları tablo olarak göster
results_df = pd.DataFrame(results, columns=["Model", "RMSE", "RMSLE"])
print("\nModel Karşılaştırma:\n", results_df)


Model Karşılaştırma:
               Model         RMSE     RMSLE
0  LinearRegression  6766.880761  1.156247
1             Ridge  6774.192060  1.154047
2      RandomForest  5548.237178  0.482309
